# Проект для «Викишоп»

## Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import WordNetLemmatizer
import re
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import (StratifiedShuffleSplit, GridSearchCV)
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import spacy

In [2]:
#!python -m spacy download en_core_web_sm

In [3]:
def regular(string):
    string = re.sub("[^a-zA-Z]"," ", str(string).lower())
    return string

def to_list(string):
    
    string = string.split()
    
    return string

def lemmatize(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    string = ""
    for token in doc:
        string += token.lemma_ + " "
    
    return re.sub("[^a-zA-Z]"," ", str(string).lower())[:-1]

def split_data(features, target):
    x_train, x_test, y_train, y_test = train_test_split(features, target,
    test_size=0.1, random_state = 12345)


    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, 
    test_size=0.15, random_state = 12345)
    
    return x_train, y_train, x_test, y_test, x_val, y_val

### Проверим работу лемматизации:

In [4]:
lemmatize("The striped bats are hanging on their feet for best")

'the stripe bat be hang on their foot for good'

### Успех!

In [5]:
#nltk.download('stopwords')

In [6]:
#nltk.download('wordnet')

In [7]:
#nltk.download('omw-1.4')

In [8]:
try:
    data = pd.read_csv('toxic_comments.csv')
except:
    data = pd.read_csv('/datasets/toxic_comments.csv')

data = data.iloc[:, 1:]
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


### Пропустим текст через функцию с регуляркой, чтобы убрать ненужные символы:

In [10]:
data['text'] = data['text'].apply(regular)
data.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m s...,0
2,hey man i m really not trying to edit war it...,0
3,more i can t make any real suggestions on im...,0
4,you sir are my hero any chance you remember...,0


### Лемматизируем

In [11]:
%%time

data['lemmatize'] = data['text'].apply(lemmatize)
data.head()

CPU times: total: 16h 6min 14s
Wall time: 16h 19min 20s


,text,toxic,lemmatize
0,explanation why the edits made under my userna...,0,explanation why the edit make under my usernam...
1,d aww he matches this background colour i m s...,0,d aww he match this background colour i m se...
2,hey man i m really not trying to edit war it...,0,hey man i m really not try to edit war it ...
3,more i can t make any real suggestions on im...,0,more i can t make any real suggestion on im...
4,you sir are my hero any chance you remember...,0,you sir be my hero any chance you rememb...


### Векторизуем текст и разделим данные на тренировочные и тестовые:

In [12]:
stop_words = set(stopwords.words('english')) 

In [13]:
corpus = data['lemmatize'].values #.astype('U')

In [14]:
x_train, y_train, x_test, y_test, x_valid, y_valid = split_data(corpus, data.toxic)

In [15]:
vectorizer = TfidfVectorizer(stop_words=list(stop_words)).fit(x_train)

x_train = vectorizer.transform(x_train)
x_valid = vectorizer.transform(x_valid)
x_test = vectorizer.transform(x_test)

### Возьмём первую модель LogisticRegression:

In [17]:
lr = LogisticRegression(random_state=12345)

cv=StratifiedShuffleSplit(n_splits=4, random_state=12345, test_size=0.1,
            train_size=None)

parameters_grid = {
    
    'C': np.arange(5, 15, 1),
    'class_weight': ['balanced'],
    'n_jobs':[-1]
}

In [18]:
%%time
lr_grid_cv = GridSearchCV(lr, parameters_grid, scoring = 'f1', cv = cv)

lr_grid_cv.fit(x_train, y_train)

CPU times: total: 3.45 s
Wall time: 2min 22s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=4, random_state=12345, test_size=0.1,
            train_size=None),
             estimator=LogisticRegression(random_state=12345),
             param_grid={'C': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                         'class_weight': ['balanced'], 'n_jobs': [-1]},
             scoring='f1')

In [19]:
print(f'Наилучшие параметры: {lr_grid_cv.best_params_}') 

f1_score(y_valid, lr_grid_cv.best_estimator_.predict(x_valid))

Наилучшие параметры: {'C': 12, 'class_weight': 'balanced', 'n_jobs': -1}


0.7571337983512998

#### Вывод: F1 на валидационных данных выше порога.

### Перейдём ко второй модели. Возьмём SGDClassifier:

In [20]:
clf = SGDClassifier(random_state=12345)

cv=StratifiedShuffleSplit(n_splits=4, random_state=12345, test_size=0.1,
            train_size=None)

parameters_grid = {
    
    'class_weight': ['balanced'],
    'penalty':["l1"],
    'l1_ratio':[0.025,.05, .1, .15, .2],
    'max_iter': range(1000, 6000, 1000)
}

In [21]:
%%time

clf_grid_cv = GridSearchCV(clf, parameters_grid, scoring = 'f1', cv = cv)

clf_grid_cv.fit(x_train, y_train)

CPU times: total: 1min 44s
Wall time: 1min 45s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=4, random_state=12345, test_size=0.1,
            train_size=None),
             estimator=SGDClassifier(random_state=12345),
             param_grid={'class_weight': ['balanced'],
                         'l1_ratio': [0.025, 0.05, 0.1, 0.15, 0.2],
                         'max_iter': range(1000, 6000, 1000),
                         'penalty': ['l1']},
             scoring='f1')

In [22]:
f1_score(y_valid, clf_grid_cv.best_estimator_.predict(x_valid))

0.725050916496945

#### Вывод: F1-мера не плохая, конечно, но не идеал.

### С хорошим запасом победила Логистическая Регрессия, испытаем её на тестовом наборе:

In [23]:
f1_score(y_test, lr_grid_cv.best_estimator_.predict(x_test))

0.769406392694064

### Общий вывод: в ходе работы была проведена лемматизация и очистка текста от ненужных символов, а так же были устранены мусорные слова. Так же были построены и обучены 2 модели, а именно LR и SGD. LR оказалась лучше. Результат на тестовой выборке для неё оказался оказался удовлетворительным для поставленной задачи.